In [16]:
import pandas as pd

# aps，重新命名了列
# aps = aps[['APSDPRDNO', 'APSDTRDAT_TM', 'APSDTRCOD', 'APSDTRAMT', 'APSDCPTPRDNO']]
# aps.columns = ['CRD_SRC', 'TRN_DT', 'TRN_COD', 'TRN_AMT', 'CRD_TGT']
# 卡号，交易日期，交易码，交易金额，对手
df = pd.read_csv('aps.csv')

# 将时间转换为datetime
df['TRN_DT'] = pd.to_datetime(df['TRN_DT'], format='%Y%m%d%H%M%S', errors='coerce')

In [ ]:
# 基准 DataFrame
def calculate_baseline(df):
    grouped = df.groupby('CRD_SRC')
    baseline = grouped.agg(
        total_transactions=('TRN_AMT', 'count'),
        total_debit_transactions=('TRN_AMT', lambda x: (x < 0).sum()),
        total_credit_transactions=('TRN_AMT', lambda x: (x > 0).sum()),
        total_amount_abs=('TRN_AMT', lambda x: x.abs().sum()),
        total_amount=('TRN_AMT', 'sum'),
        total_debit_amount=('TRN_AMT', lambda x: x[x < 0].sum()),
        total_credit_amount=('TRN_AMT', lambda x: x[x > 0].sum()),
        unique_counterparties=('CRD_TGT', 'nunique'),
        unique_debit_counterparties=('CRD_TGT', lambda x: x[df['TRN_AMT'] < 0].nunique()),
        unique_credit_counterparties=('CRD_TGT', lambda x: x[df['TRN_AMT'] > 0].nunique())
    ).reset_index()
    return baseline


# 计算基准值
baseline_df = calculate_baseline(df)

In [ ]:
# 按小时计算并与基准值相除
def calculate_hourly_ratios(df, baseline_df):
    df['hour'] = df['TRN_DT'].dt.hour
    hourly_grouped = df.groupby(['CRD_SRC', 'hour'])
    hourly = hourly_grouped.agg(
        total_transactions=('TRN_AMT', 'count'),
        total_debit_transactions=('TRN_AMT', lambda x: (x < 0).sum()),
        total_credit_transactions=('TRN_AMT', lambda x: (x > 0).sum()),
        total_amount_abs=('TRN_AMT', lambda x: x.abs().sum()),
        total_amount=('TRN_AMT', 'sum'),
        total_debit_amount=('TRN_AMT', lambda x: x[x < 0].sum()),
        total_credit_amount=('TRN_AMT', lambda x: x[x > 0].sum()),
        unique_counterparties=('CRD_TGT', 'nunique'),
        unique_debit_counterparties=('CRD_TGT', lambda x: x[df['TRN_AMT'] < 0].nunique()),
        unique_credit_counterparties=('CRD_TGT', lambda x: x[df['TRN_AMT'] > 0].nunique())
    ).reset_index()

    ratios = pd.merge(hourly, baseline_df, on='CRD_SRC', suffixes=('_hourly', '_baseline'))

    for col in ['total_transactions', 'total_debit_transactions', 'total_credit_transactions', 'total_amount_abs',
                'total_amount', 'total_debit_amount', 'total_credit_amount', 'unique_counterparties',
                'unique_debit_counterparties', 'unique_credit_counterparties']:
        ratios[col + '_ratio'] = (ratios[col + '_hourly'] / ratios[col + '_baseline']).round(2)

    hourly_ratios = ratios.pivot(index='CRD_SRC', columns='hour')
    hourly_ratios.columns = ['_'.join(map(str, col)).strip() for col in hourly_ratios.columns.values]
    hourly_ratios.reset_index(inplace=True)

    return hourly_ratios


hourly_ratios = calculate_hourly_ratios(df, baseline_df)

In [ ]:
hourly_ratios.head(5)

In [6]:
# 按星期计算并与基准值相除
def calculate_weekly_ratios(df, baseline_df):
    df['weekday'] = df['TRN_DT'].dt.weekday + 1
    weekly_grouped = df.groupby(['CRD_SRC', 'weekday'])
    weekly = weekly_grouped.agg(
        total_transactions=('TRN_AMT', 'count'),
        total_debit_transactions=('TRN_AMT', lambda x: (x < 0).sum()),
        total_credit_transactions=('TRN_AMT', lambda x: (x > 0).sum()),
        total_amount_abs=('TRN_AMT', lambda x: x.abs().sum()),
        total_amount=('TRN_AMT', 'sum'),
        total_debit_amount=('TRN_AMT', lambda x: x[x < 0].sum()),
        total_credit_amount=('TRN_AMT', lambda x: x[x > 0].sum()),
        unique_counterparties=('CRD_TGT', 'nunique'),
        unique_debit_counterparties=('CRD_TGT', lambda x: x[df['TRN_AMT'] < 0].nunique()),
        unique_credit_counterparties=('CRD_TGT', lambda x: x[df['TRN_AMT'] > 0].nunique())
    ).reset_index()

    ratios = pd.merge(weekly, baseline_df, on='CRD_SRC', suffixes=('_weekly', '_baseline'))

    for col in ['total_transactions', 'total_debit_transactions', 'total_credit_transactions', 'total_amount_abs',
                'total_amount', 'total_debit_amount', 'total_credit_amount', 'unique_counterparties',
                'unique_debit_counterparties', 'unique_credit_counterparties']:
        ratios[col + '_ratio'] = (ratios[col + '_weekly'] / ratios[col + '_baseline']).round(2)

    weekly_ratios = ratios.pivot(index='CRD_SRC', columns='weekday')
    weekly_ratios.columns = ['_'.join(map(str, col)).strip() for col in weekly_ratios.columns.values]
    weekly_ratios.reset_index(inplace=True)

    return weekly_ratios


weekly_ratios = calculate_weekly_ratios(df, baseline_df)

In [10]:
# 特定时段统计并与基准值相除
def calculate_specific_period_ratios(df, baseline_df):
    specific_periods = {
        'night_11_to_1': (23, 1),
        'night_0_to_3': (0, 3),
        'night_11_to_6': (23, 6)
    }

    results = []

    for label, (start_hour, end_hour) in specific_periods.items():
        if start_hour < end_hour:
            period_condition = (df['TRN_DT'].dt.hour >= start_hour) & (df['TRN_DT'].dt.hour < end_hour)
        else:  # Handle cross-midnight periods
            period_condition = (df['TRN_DT'].dt.hour >= start_hour) | (df['TRN_DT'].dt.hour < end_hour)

        df_period = df[period_condition]

        period_grouped = df_period.groupby('CRD_SRC')
        period = period_grouped.agg(
            total_transactions=('TRN_AMT', 'count'),
            total_debit_transactions=('TRN_AMT', lambda x: (x < 0).sum()),
            total_credit_transactions=('TRN_AMT', lambda x: (x > 0).sum()),
            total_amount_abs=('TRN_AMT', lambda x: x.abs().sum()),
            total_amount=('TRN_AMT', 'sum'),
            total_debit_amount=('TRN_AMT', lambda x: x[x < 0].sum()),
            total_credit_amount=('TRN_AMT', lambda x: x[x > 0].sum()),
            unique_counterparties=('CRD_TGT', 'nunique'),
            unique_debit_counterparties=('CRD_TGT', lambda x: x[df['TRN_AMT'] < 0].nunique()),
            unique_credit_counterparties=('CRD_TGT', lambda x: x[df['TRN_AMT'] > 0].nunique())
        ).reset_index()

        period_ratios = pd.merge(period, baseline_df, on='CRD_SRC', suffixes=('_period', '_baseline'))

        for col in ['total_transactions', 'total_debit_transactions', 'total_credit_transactions', 'total_amount_abs',
                    'total_amount', 'total_debit_amount', 'total_credit_amount', 'unique_counterparties',
                    'unique_debit_counterparties', 'unique_credit_counterparties']:
            period_ratios[col + '_ratio'] = (period_ratios[col + '_period'] / period_ratios[col + '_baseline']).round(2)

        period_ratios['period'] = label
        results.append(period_ratios)

    specific_period_ratios = pd.concat(results, ignore_index=True)
    specific_period_pivot = specific_period_ratios.pivot(index='CRD_SRC', columns='period')
    specific_period_pivot.columns = ['_'.join(map(str, col)).strip() for col in specific_period_pivot.columns.values]
    specific_period_pivot.reset_index(inplace=True)

    return specific_period_pivot


specific_period_ratios = calculate_specific_period_ratios(df, baseline_df)

C:\Users\zangq\AppData\Local\Temp\ipykernel_19020\3488553419.py:43: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  specific_period_ratios = pd.concat(results, ignore_index=True)


In [11]:
# 合并所有结果
final_df = baseline_df
final_df = pd.merge(final_df, hourly_ratios, on='CRD_SRC', how='left')
final_df = pd.merge(final_df, weekly_ratios, on='CRD_SRC', how='left')
final_df = pd.merge(final_df, specific_period_ratios, on='CRD_SRC', how='left')

# 保存结果
final_df.to_csv('time_series_2.csv', index=False)